# Algorithm B

### Imports

In [1]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from stldecompose import decompose
from statsmodels.tsa.stattools import adfuller
import logging
from sklearn.metrics import mean_squared_error

### Test Stationarity function definition

In [2]:
def test_stationarity(timeseries):
    """Performs  Dickey-Fuller test to test stationairty"""
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

### Anchor Function definition

In [3]:
def anchor(signal, weight):
    """Data Smooothing"""
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

### Abline Function Definition

In [4]:
def abline(slope, intercept):
    """Plots a line from slope and intercept"""
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '--')

### Strongest Trend Period Definition

In [5]:
def strongest_trend_period(ts, start, end):
    """Trend Detection, ts shape(x, )"""
    trend_strengths = []
    for i in range(start, end):
        decomposition = decompose(ts, period = i)
        var_residual = np.power(st.stdev(decomposition.resid), 2)
        var_trend_residual = np.power(st.stdev(anchor(decomposition.trend, 0.9) + decomposition.resid), 2)
        trend_strength = max(0, (1 - var_residual/var_trend_residual))
        trend_strengths.append(trend_strength)
    d = dict()
    d["index"] = trend_strengths.index(max(trend_strengths))
    d["period"] = d["index"] + 1
    d["trend_strength"] = max(trend_strengths)
    return d

### Strongest Seasonal Period Definition

In [6]:
def strongest_seasonal_period(ts, start, end):
    """Period Detecion, tx shape (x, )"""
    seasonal_strengths = []
    for i in range(start, end):
        decomposition = decompose(ts, period = i)
        var_residual = np.power(st.stdev(decomposition.resid), 2)
        var_seasonal_residual = np.power(st.stdev(decomposition.seasonal + decomposition.resid), 2)
        seasonality_strength = max(0, (1 - var_residual/var_seasonal_residual))
        seasonal_strengths.append(seasonality_strength)
    d = dict()
    d["index"] = seasonal_strengths.index(max(seasonal_strengths))
    d["period"] = d["index"] + 1
    d["seasonality_strength"] = max(seasonal_strengths)
    return d

### Remove Trend Definition

In [7]:
def remove_trend(ts, p):
    """Remove Trend, ts shape (x, )"""
    decomposition = decompose(ts, period = p)
    detrended = ts - decomposition.trend
    d = dict()
    d["detrended"] = detrended.reshape(detrended.shape[0], 1)
    d["trend"] = decomposition.trend.reshape(decomposition.trend.shape[0], 1)
    return d

### Linearity Score Definition

In [8]:
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
def linearity_score(ts):
    """Linarity Score Detection"""
    x = [e for e in range(0, ts.shape[0])]
    x = np.array(x)
    trend_train_df = pd.DataFrame({'time': x, 'trend':ts})
    model_trend_train = smf.ols('time ~ trend', data = trend_train_df).fit()
    return model_trend_train.rsquared

### Scale Definition (-1, 1)

In [9]:
def scale(ts):
    """Scaling data, -1 to 1, ts shape (x, )"""
    scaler_ts = MinMaxScaler(feature_range = (-1, 1))
    scaler_ts = scaler_ts.fit(ts.values.reshape(-1, 1))
    ts_scaled = scaler_ts.transform(ts.values.reshape(-1, 1))
    ts_scaled = pd.DataFrame(ts_scaled)
    d = dict()
    d["scaler"] = scaler_ts
    d["scaled"] = ts_scaled #(x, 1)
    return d

### Linear Reg Definction

In [10]:
def linear_reg(train, start, end):
    """Performs Linear Regression, train shape (x, )"""
    x = [e for e in range(0, train.shape[0])]
    x = np.array(x)
    x_test = [e for e in range(start, end)]
    x_test = np.array(x_test)
    linearRegressor = LinearRegression()
    linearRegressor.fit(x.reshape(-1, 1), train.reshape(-1, 1))
    # model_trend_train.predict(z)
    pred = linearRegressor.predict(x_test.reshape(-1, 1))
    return pred

### Configuration for Figure Size

In [11]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 20
fig_size[1] = 7

### Data Generation By ARMA

In [12]:
# ARMA example
# Data creation, random data is created, ARMA is applied, its fitted values are usede as a TS
from statsmodels.tsa.arima_model import ARMA
import random

data = [random.randint(1, 100) for x in range(0, 100000)]
model = ARMA(data, order=(0, 1))
model_fit = model.fit(disp=False)
plt.plot(data[:1000])
test_stationarity(pd.Series(model_fit.fittedvalues))
Test Statistic is smaller than vritical value => TS is stationary 
ts = model_fit.fittedvalues
print(type(ts))
plt.plot(ts[:100])

np.savetxt("stationary-arma-ts.csv", ts, delimiter=",", header = "Data")

### Trendy Data Generation

In [13]:
import random
import math
from statsmodels.tsa.arima_model import ARMA

x = [e for e in range(0, 18000)]
noise = [5*random.random() for i in range(0, 18000)]
y = []
for j in range(len(x)):
    y.append(x[j])
y = np.array(y)
data = y + noise
plt.plot(data[:50])
print(data)

np.savetxt("trendy-ts-18000.csv", data, delimiter=",", header = "Data")

### Periodic Data Generation

In [14]:
import random
import math
from statsmodels.tsa.arima_model import ARMA

x = [e for e in range(0, 1300)]
noise = [random.random() for i in range(0, 1300)]
x = np.arange(0, 195, 0.15);
y = np.sin(x)
scaler_y = MinMaxScaler(feature_range = (-1, 1))
scaler_y = scaler_y.fit(y.reshape(-1, 1))
y_scaled = scaler_y.transform(ts.values.reshape(-1, 1))
y = y_scaled.flatten()
data = y + noise
plt.plot(data[:200])

np.savetxt("seasonal-ts-sin.csv", data, delimiter=",", header = "Data")

### Reading the data

In [3]:
#Stationry-Arma-Ts
#Trendy-Ts
#Seasonal-Ts
df = pd.read_csv("trendy-ts-18000.csv")
ts = df["# Data"]
ts = ts[:18000]
print(df['# Data'].describe())
print("\n")
print("Shape of TS: " + str(ts.shape))
print("Type of TS: " + str(type(ts)))

### Scaling data to (-1, 1) range

In [17]:
ts_scaler = scale(ts)
ts_scaled = ts_scaler["scaled"]
batch_size = 25

In [18]:
from __future__ import print_function
import matplotlib as mpl
from matplotlib import pyplot as plt
import mxnet as mx
from mxnet import gluon, autograd, nd
from mxnet.gluon import nn, rnn

ctx = mx.cpu()

### Spliting the data into train (70%) and test (30%) sets

In [5]:
whole_data_size = ts_scaled.shape[0]
train_data_size = int((whole_data_size*70)/100)
test_data_size = int((whole_data_size*30)/100)

ts_scaled = ts_scaled.values
train = ts_scaled[:train_data_size]
test = ts_scaled[train_data_size:]

print("Train set shape: " + str(train.shape))
print("Test set shape: " + str(test.shape))
print("\n")

train_sd = st.stdev(train.reshape(train_data_size, ))
train_mean = st.mean(train.reshape(train_data_size, ))
print("Train SD: " + str(train_sd))
print("Train Mean: " + str(train_mean))

ts_test = test
ts_train = train

### Trend and Seasonality
##### Scaling for Trend and Detrended

In [23]:
#For Train
str_trd_train = strongest_trend_period(ts_train.reshape(ts_train.shape[0], ), 1, 25)
str_trd_train["period"]

detrended_train = remove_trend(ts_train.reshape(ts_train.shape[0], ), str_trd_train["period"])
if(linearity_score(detrended_train["trend"].reshape(detrended_train["trend"].shape[0], )) >= 0.8):
    ts_train = detrended_train["detrended"]
    ts_detrended_train_scaler = scale(pd.Series(ts_train.reshape(ts_train.shape[0], )))
    ts_train = ts_detrended_train_scaler["scaled"].values

str_period_train = strongest_seasonal_period(ts_train.reshape(ts_train.shape[0], ), 1, 25)
if(str_period_train["seasonality_strength"] >= 0.8):
    batch_size = str_period_train["period"]

#For Test

str_trd_test = strongest_trend_period(ts_test.reshape(ts_test.shape[0], ), 1, 25)
str_trd_test["period"]
detrended_test = remove_trend(ts_test.reshape(ts_test.shape[0], ), str_trd_test["period"])

if(linearity_score(detrended_test["trend"].reshape(detrended_test["trend"].shape[0], )) >= 0.8):
    ts_test = detrended_test["detrended"]
    ts_detrended_test_scaler = scale(pd.Series(ts_test.reshape(ts_test.shape[0], )))
    ts_test = ts_detrended_test_scaler["scaled"].values

### Network Architecture

In [25]:
#Generator
netG = nn.Sequential()
with netG.name_scope():
    netG.add(nn.Dense(batch_size))
    netG.add(nn.BatchNorm(momentum = 0.8))
    netG.add(nn.Dropout(0.5))
    netG.add(nn.Dense(batch_size+15))
    netG.add(nn.BatchNorm(momentum = 0.8))
    netG.add(nn.Dropout(0.5))
    netG.add(nn.Dense(2*(batch_size+15)))
    netG.add(nn.BatchNorm(momentum = 0.8))
    netG.add(nn.Dropout(0.5))
    netG.add(nn.Dense(1, activation = "tanh"))

#Discriminator
netD = nn.Sequential()
with netD.name_scope():
    netD.add(nn.Dense(batch_size, activation = "tanh"))
    netD.add(nn.Dense(15, activation = "tanh"))
    netD.add(nn.Dense(10, activation='tanh'))
    netD.add(nn.Dense(10, activation='tanh'))
    netD.add(nn.Dense(5, activation='tanh'))
    netD.add(nn.Dense(batch_size))
    
# print(netG)
# print(netD)    

# loss
loss = gluon.loss.SoftmaxCrossEntropyLoss()

#initialize the generator and the discriminator
#assigning weights from Normal distribution 
netG.initialize(mx.init.Normal(0.02), ctx=ctx)
netD.initialize(mx.init.Normal(0.02), ctx=ctx)

#trainer for the generator and the discriminator
#appling an Optimazer to a set of parmas, takes the params of netG/D, and uses adam opt
trainerG = gluon.Trainer(netG.collect_params(), 'adam', {'learning_rate': 0.01})
trainerD = gluon.Trainer(netD.collect_params(), 'adam', {'learning_rate': 0.05})

In [27]:
#ts shape (x, 1)
# set up logging
from datetime import datetime
import os
import time

def train_loop(ts, batch_size, epochs):
    Y = nd.ones(shape=(ts.shape[0], 1))
    train_data = mx.io.NDArrayIter(ts, Y, batch_size, shuffle=False)

    real_label = mx.nd.ones((batch_size, ), ctx=ctx)
    fake_label = mx.nd.zeros((batch_size, ), ctx=ctx)
    metric = mx.metric.Accuracy()

    stamp =  datetime.now().strftime('%Y_%m_%d-%H_%M')
    logging.basicConfig(level=logging.DEBUG)
    
    g_loss = []
    d_loss = []
    mse = []

    for epoch in range(epochs):
        tic = time.time()
        train_data.reset()
        iter = 0
        for i, batch in enumerate(train_data):
            # (1) Updating D network: maximize log(D(x)) + log(1 - D(G(z)))
            data = batch.data[0].as_in_context(ctx)
            noise = mx.nd.array(ts)
            noise = noise[(i*batch_size):((i+1)*batch_size), 0]

            with autograd.record():
                real_output = netD(data)
                errD_real = loss(real_output, real_label)

                fake = netG(noise)
                fake_output = netD(fake.detach())
                errD_fake = loss(fake_output, fake_label)
                errD = errD_real + errD_fake
                errD.backward()

            trainerD.step(batch_size)
            metric.update([real_label,], [real_output,])
            metric.update([fake_label,], [fake_output,])
            
            #Updating G network: maximize log(D(G(z)))
            with autograd.record():
                output = netD(fake)
                errG = loss(output, real_label)
                errG.backward()

            trainerG.step(batch_size)

        name, acc = metric.get()
        metric.reset()
        print('\nbinary training acc at epoch %d: %s=%f' % (epoch, name, acc))
        print('time: %f' % (time.time() - tic))

        if iter % 1024 == 0:
            logging.info('discriminator loss = %f, generator loss = %f, at iter %d epoch %d' 
                 %(nd.mean(errD).asscalar(), 
                   nd.mean(errG).asscalar(), iter, epoch))
            d_loss.append(nd.mean(errD).asscalar())
            g_loss.append(nd.mean(errG).asscalar())
        iter = iter + 1

        noise = mx.nd.array(ts)

        noise = noise[:, 0]
        fake = netG(noise)
        plt.plot(ts[:200, ], color = "#007d92", label = "Train")
        plt.plot(fake[:200, 0].asnumpy(), color = "#cd8fa3", label = "Generated")
        plt.xlabel("Time")
        plt.legend(loc = "upper left")
        plt.show()

        print("Error Rate: " + str(mean_squared_error(ts, fake.asnumpy())))
        mse.append(mean_squared_error(ts, fake.asnumpy()))
    
    #plotting behovior
    plt.plot(d_loss, color = "#cd8fa3", label = "Discriminator Loss ")
    plt.legend(loc = "upper right")
    plt.xlabel("Number of Epochs")
    plt.ylabel("Discriminator Loss")
    plt.title("Behavior of Discriminator Loss")
    plt.show()

    plt.plot(g_loss, color = "#f3166b", label = "Generator Loss")
    plt.legend(loc = "upper right")
    plt.xlabel("Number of Epochs")
    plt.ylabel("Generator Loss")
    plt.title("Behavior of Generator Loss")
    plt.show()

    plt.plot(g_loss, color = "#007d92", label = "MSE")
    plt.legend(loc = "upper right")
    plt.xlabel("Number of Epochs")
    plt.ylabel("MSE")
    plt.title("Behavior of MSE")
    plt.show()

In [4]:
train_loop(ts_train, batch_size, 50)

In [6]:
#last part of train
noise = mx.nd.array(ts_train[ts_train.shape[0]-batch_size:])
fake = netG(noise)
fake = fake.asnumpy()
plt.plot(ts_test[:, ], color = "#007d92", label = "Test")
plt.plot(fake[:, ],  color = "#cd8fa3", label = "Generated")
plt.legend(loc = "upper left")
plt.show()
print(mean_squared_error(ts_test[:batch_size, 0], fake[:, 0]))

np.savetxt(("Version_B//Stationary//" + str(batch_size) + "-" + "00" + ".csv"), fake[:, 0], delimiter=",", header="Data")

for i in range(int(ts_test.shape[0]/batch_size)-1):
    noise = mx.nd.array(ts_test)
    noise = noise[batch_size * (i) :batch_size * (1 + i), 0]
    fake = netG(noise)
    fake = fake.asnumpy()

    for x in range(batch_size * (1+i)):
        fake = np.insert(fake, x, None, axis=0)
    #plot
    plt.plot(ts_test[:, ], color = "#007d92", label = "Test")
    plt.plot(fake[:, ],  color = "#cd8fa3", label = "Generated")
    plt.legend(loc = "upper left")
    plt.show()
    
    print(mean_squared_error(ts_test[batch_size * (i+1):batch_size * (2 + i), 0], fake[batch_size * (i+1):, 0]))

    np.savetxt(("Version_B//Stationary//" + str(batch_size) + "-" + str(i) + ".csv"), fake[:, 0], delimiter=",", header="Data")

### Testing Process

In [7]:
fake_all = []
f = pd.read_csv("Version_B//Stationary//" + str(batch_size) + "-00" + ".csv")
f = f["# Data"].values
for x in range(f.shape[0]):
    fake_all.append(f[x])

fake_all

for i in range(0, int(ts_test.shape[0]/batch_size)-1):
    f = pd.read_csv("Version_B//Stationary//" + str(batch_size) + "-" + str(i) + ".csv")
    f = f["# Data"].dropna().values
    for x in range(f.shape[0]):
        fake_all.append(f[x])

fake_all = np.array(fake_all)
fake_all = fake_all.reshape(fake_all.shape[0], 1)
plt.plot(ts_test[:200, 0], color = "#007d92", label = "Test")
plt.plot(fake_all[:200, 0],  color = "#cd8fa3", label = "Generated")
plt.legend(loc = "upper left")
plt.xlabel("Time")
plt.show()

print(mean_squared_error(ts_test, fake_all))

np.savetxt(("Version_B//Stationary//" + "fake_all-" + str(batch_size) + ".csv"), fake_all[:, 0], delimiter=",", header = 'Data')

### Bringing the Trendy Back

In [8]:
diff = []

generated_inverese_scaled = ts_detrended_test_scaler["scaler"].inverse_transform(fake_all)
ts_test_original = ts_scaled[ts_scaled.shape[0] - ts_test.shape[0]:]
with_trend = generated_inverese_scaled + linear_reg(detrended_train["trend"].reshape(detrended_train["trend"].shape[0], ), detrended_train["trend"].shape[0], detrended_train["trend"].shape[0] + detrended_test["trend"].shape[0])

gen_rescaled = ts_scaler["scaler"].inverse_transform(generated_inverese_scaled)
ts_test_rescaled = ts_scaler["scaler"].inverse_transform(ts_test_original)
with_trend_rescaled = ts_scaler["scaler"].inverse_transform(with_trend)

ts_test_original_list = ts_test_rescaled[:, ].reshape(ts_test_rescaled.shape[0], )
with_trend_list = with_trend_rescaled[:, ].reshape(with_trend_rescaled.shape[0], )

for u in range(len(ts_test_original_list[:, ])):
    diff.append(abs(ts_test_original_list[u]-with_trend_list[u]))

residual_stdev = st.stdev(diff)
ub = with_trend_rescaled[:, 0] + (1.96*residual_stdev)
lb = with_trend_rescaled[:, 0] - (1.96*residual_stdev)

plt.plot(ts_test_rescaled[:50], color = "#007d92", label = "Test")
#plt.plot(with_trend_rescaled[:50], color = "#cd8fa3", label = "Generated")
plt.fill_between(x = df.index[:50], y1 = ub[:50, ], 
                 y2 = lb[:50, ], color = "gainsboro",
                 label = "Confidence Bounds") 
plt.legend(loc = "upper left")
plt.xlabel("Time")
plt.show()

### Anomaly detection

In [9]:
import math

fake_all = pd.read_csv("Version_B//Stationary//fake_all-" + str(batch_size) + ".csv")
fake_all = fake_all["# Data"].values.reshape(fake_all.shape[0], 1)

diff = []
anomaly_indices = []

# test_smoothed[100] = 1.5
# test_smoothed[110] = 2
# test_smoothed[150] = 2

ts_test_rescaled = ts_scaler["scaler"].inverse_transform(ts_test[:, ])[:, 0]
fake_rescaled = ts_scaler["scaler"].inverse_transform(fake_all[:, ])

ts_test_list = ts_test_rescaled[:, ].reshape(ts_test_rescaled.shape[0], )
generated_list = fake_rescaled[:, ].reshape(fake_rescaled.shape[0], )

for u in range(len(ts_test_list[:, ])):
    diff.append(abs(ts_test_list[u]-generated_list[u]))

diff_list_df = pd.DataFrame({"residual": diff, 
                            "predicted": generated_list})
diff_list_df.head()

residual_mean = st.mean(diff)
residual_stdev = st.stdev(diff)
upper_bound = residual_mean + (3*residual_stdev)
lower_bound = residual_mean - (3*residual_stdev)
plt.scatter(x = generated_list, y = diff, color = "#cd8fa3")
#plt.plot(diff[:200])
abline(0, upper_bound)
abline(0, lower_bound)
plt.xlabel("Generated")
plt.ylabel("Residual")
plt.title("Generated vs Residual")
plt.show()


### Rescaled CI for the Generated Data

In [10]:
ts_test_rescaled = ts_scaler["scaler"].inverse_transform(ts_test[:, ])[:, 0]
ub_rescaled = ts_scaler["scaler"].inverse_transform(fake_all[:, ] + (1.96*residual_stdev) + residual_mean)
lb_rescaled = ts_scaler["scaler"].inverse_transform(fake_all[:, ] - (1.96*residual_stdev) + residual_mean)

plt.plot(ts_test_rescaled[:200, ], color = "#007d92", label = "Test")
# plt.plot(fake_all[batch_size:200, ] + (1.96*residual_stdev), "--", color = "grey", label = "Upper Bound")
# plt.plot(fake_all[batch_size:200, ] - (1.96*residual_stdev), "--", color = "pink", label = "Lower Bound")

plt.plot(ts_scaler["scaler"].inverse_transform(fake_all[:200, ])[:, 0],  color = "#cd8fa3", label = "Generated")

# plt.fill_between(x = df.index[:200], y1 = ub_rescaled[:200, 0], 
#                  y2 = lb_rescaled[:200, 0], color = "gainsboro", 
#                  label = "Confidence Bounds")
plt.legend(loc = "upper left")
plt.xlabel("Time")
plt.show()


### Avergaed Version and Confidence Bounds

In [11]:
fake_10_50 = pd.read_csv("Version_B//Stationary//fake_all-10.csv")
fake_10_50 = fake_10_50["# Data"].values

fake_20_50 = pd.read_csv("Version_B//Stationary//fake_all-20.csv")
fake_20_50 = fake_20_50["# Data"].values

fake_25_50 = pd.read_csv("Version_B//Stationary//fake_all-25.csv")
fake_25_50 = fake_25_50["# Data"].values

fake_30_50 = pd.read_csv("Version_B//Stationary//fake_all-30.csv")
fake_30_50 = fake_30_50["# Data"].values

fake_36_50 = pd.read_csv("Version_B//Stationary//fake_all-36.csv")
fake_36_50 = fake_36_50["# Data"].values

fake_mean_all = (fake_10_50 +  fake_36_50 + fake_30_50)/3
diff = []

ts_test_rescaled = ts_scaler["scaler"].inverse_transform(ts_test[:, ])
fake_mean_all_rescaled = ts_scaler["scaler"].inverse_transform(fake_mean_all.reshape(fake_mean_all.shape[0], 1)[:, ])

ts_test_rescaled_list = ts_test_rescaled[:, ].reshape(ts_test_rescaled.shape[0], )
generated_mean_list = fake_mean_all_rescaled[:, ].reshape(ts_test_rescaled.shape[0], )

for u in range(len(ts_test_rescaled_list[:, ])):
    diff.append(abs(ts_test_rescaled_list[u]-generated_mean_list[u]))

diff_list_df = pd.DataFrame({"residual": diff, 
                            "predicted": generated_mean_list})

residual_mean = st.mean(diff)
residual_stdev = st.stdev(diff)

ub = fake_mean_all_rescaled + (1.96*residual_stdev)
lb = fake_mean_all_rescaled - (1.96*residual_stdev)

plt.plot(ts_test_rescaled[0:200, 0], color = "#007d92", label = "Test")
plt.plot(fake_mean_all_rescaled[:200, ],  color = "#cd8fa3", label = "Generated Mean")

# plt.fill_between(x = df.index[:200], y1 = ub[:200, 0], 
#                  y2 = lb[:200, 0], color = "gainsboro",
#                  label = "Confidence Bounds For Mean")

plt.legend(loc = "upper left")
plt.xlabel("Time")
plt.show()

print("Test Error Rate: " + str(mean_squared_error(ts_test, fake_mean_all)))